In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import os
os.environ['HOME_DIR'] = 'drive/MyDrive/hidden-networks'
!pip install -r $HOME_DIR/requirements.txt

import sys
sys.path.append(os.path.join('/content', os.environ['HOME_DIR']))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.autograd as autograd
import collections
import numpy as np
from scipy.integrate import quad, dblquad
from scipy.stats import hypsecant
from scipy.stats import uniform, norm
import math

# from signal_supermask_pruning2 import GetSubnet, SupermaskConv, SupermaskLinear
# from signal_supermask_pruning2 import train, test
from jeffrey_utils import *

In [ ]:
train_args = {
  "lr": 0.001, # learning rate (default: 0.1)
  "momentum": 0.9, # Momentum (default: 0.9)
  'wd': 0.001, # Weight decay (default: 0.0005)
  'epochs': 99,
  'log_interval': 100, # how many batches to wait before logging training status
  'train_eval_interval': None, # epoch interval at which to print training accuracy
  'test_eval_interval': 1, # epoch interval at which to print test accuracy
  'eval_train_on_last': False,
  'eval_test_on_last': True,
  'alpha': 0
}

In [ ]:
train_after(trained_model, train_args, device, train_loader, test_loader)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Train Epoch: 1 [0/50000 (0%)]	Loss: 1.566135
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.411218
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.382881
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.490156

Test set: Average loss: 0.0016, Accuracy: 4512/10000 (45%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.397229
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.544092
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.369242
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.424219

Test set: Average loss: 0.0016, Accuracy: 4563/10000 (46%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 1.397162
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.257518
Train Epoch: 3 [25600/50000 (51%)]	Loss: 1.286005
Train Epoch: 3 [38400/50000 (77%)]	Loss: 1.362649

Test set: Average loss: 0.0016, Accuracy: 4573/10000 (46%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 1.265250
Train Epoch: 4 [12800/50000 (26%)]	Loss: 1.233904
Train Epoch: 4 [25600/50000 (51%)]	Loss: 1.311176
Train Epoch: 4 [38400/50000 (77%)]	Loss: 1.304998

Test set: Average loss: 0.0016, Accu

KeyboardInterrupt: ignored

In [ ]:
# Arguments that do not affect model at all
num_layers = 2
sparsity = 0.5
layer_size = 7000

model_args = {
  "dataset": "CIFAR10",
  "batch_size": 128, # input batch size for training (default: 64)
  "epochs": 1, # number of epochs to train (default: 14)
  "lr": 0.1, # learning rate (default: 0.1)
  "momentum": 0.9, # Momentum (default: 0.9)
  'wd': 1e-4, # Weight decay (default: 0.0005)
  'no_cuda': False, # disables CUDA training
  'seed': 69, # random seed (default: 1)
  'save_name': None, # "simple20_rs2", # For Saving the current Model, None if not saving
  'sparsity': [sparsity] * num_layers, # 'how sparse is each layer'
  'copy_layers': [], # ['conv1', 'conv2', 'fc2'],
  'freeze_layers': [],
  'num_layers': num_layers,
  'layer_size': layer_size,
  'input_size': 32 * 32 * 3
}

prune_args = {
    "test_batch_size": 1000, # input batch size for testing (default: 1000)
    'data': '../data', # Location to store data (e.g. MNIST)
    'log_interval': 100, # how many batches to wait before logging training status
    'train_eval_interval': None, # epoch interval at which to print training accuracy
    'test_eval_interval': 1, # epoch interval at which to print test accuracy
    'eval_train_on_last': False,
    'eval_test_on_last': True,
    'eval_on_last': True,
    'alpha': 1e-2
}

train_args = {
  "lr": 0.001, # learning rate (default: 0.1)
  "momentum": 0.9, # Momentum (default: 0.9)
  'wd': 0.001, # Weight decay (default: 0.0005)
  'epochs': 1,
  'log_interval': 100, # how many batches to wait before logging training status
  'train_eval_interval': None, # epoch interval at which to print training accuracy
  'test_eval_interval': 1, # epoch interval at which to print test accuracy
  'eval_train_on_last': False,
  'eval_test_on_last': True,
  'alpha': 0
}



pruned_model, device, train_loader, test_loader, criterion = main(model_args, prune_args)

Using device cuda
Files already downloaded and verified


KeyboardInterrupt: ignored

In [ ]:
weights = pruned_model.get_unpruned_weights()

In [ ]:
def isometry_penalty(weights, gain=math.sqrt(2)):
  penalty = 0
  for weight in weights:
    dot = weight.T @ weight / gain ** 2
    penalty += torch.norm(dot - torch.eye(dot.shape[0]).to('cuda'))
  return penalty

In [ ]:
isometry_penalty(weights)

tensor(62.6738, device='cuda:0')